# Day 5

## Problem 1

Here, we're parsing a list of ordering rules, and then checking to see if a sequence follows all the rules

Cant think of a good approach beyond just:

- parse the list of rules into a dict. Key is number, value is list of rules its involved in.
- for each rule, place it in two lists, one for the first number, one for the second number
- for each sequece, get list of rules that pertain to it. (into set so its unique)
- for each rule, scan sequence to see if it follows it.


Not happy about this, there has to be a better way... 

I was thinking of just assigning characters/emojis to the numbers and the rules just specify an ordering, and then trying to create a minheap according to this ordering, etc. Unclear, just vague thoughts right now.

An idea is to treat the rules as a directed graph. Each rule is a directed edge from the first number to the second number. 

**Note**: This is a topological sorting problem - weak area for me

**Note** - a key observation is that the rules are provided for every value AFTER a number. We arent meant to interpret them transitively. So if we have a rule, like `75|47`  and `47|61`, we dont need to infer that `75|61`. This is also given. 

This reminds me of stuff from discrete math that I cant now fully remember....

Anyway, this lets us just check each value in the sequence against the rules where its on the LHS.

In [1]:
from collections import defaultdict


def form_input(file):
    rules, sequences = [], []

    with open(file) as f:
        file = f.read()

    rules, sequences = file.split("\n\n")
    # re-form rules
    rules = rules.split("\n")
    # re-form sequences
    sequences = [list(map(int, s.split(","))) for s in sequences.split("\n")]

    # form graph
    graph = defaultdict(list)
    for rule in rules:
        # a before b
        a, b = map(int, rule.split("|"))
        graph[a].append(b)

    return rules, graph, sequences


def validate(sequence, graph):
    # for each page in the sequence
    for idx, page_num in enumerate(sequence):
        # iterate over the pages that come after it and make sure
        for next_page in sequence[idx + 1 :]:
            if next_page not in graph[page_num]:
                return None
    # all rules pass on this sequence
    return sequence[len(sequence) // 2]


def p1(sequences, graph):
    sum = 0
    for sequence in sequences:
        if mid_num := validate(sequence, graph):
            sum += mid_num
    return sum

### Example

In [2]:
rules, graph, sequences = form_input("./data/day5_small_input_p1.txt")
p1(sequences, graph)

143

### Full Problem


In [3]:
rules, graph, sequences = form_input("./data/day5_input.txt")
p1(sequences, graph)

6041

### Optimization

This works, but of course is not ideal - Im going through subsequences for each element in the sequence! 

Another (better?) approach, is to iterate over the graph itself:

In [4]:
def validate_by_graph(sequence, graph):
    # precompute the position of each page in the sequence
    position = {page: idx for idx, page in enumerate(sequence)}

    # check all nodes in the graph (rules)
    for node in graph:
        for outgoing_node in graph[node]:
            # if both this node and the outgoing_node are in the sequence
            if node in position and outgoing_node in position:
                # validate their order
                if position[node] > position[outgoing_node]:
                    return None  # invalid order

    # If all edges are valid, return the middle page
    return sequence[len(sequence) // 2]

Lets take a quick look:

In [5]:
graph = {76: [23, 47], 23: [47], 47: []}
print(validate_by_graph(sequence=[76, 23, 47], graph=graph))
print(validate_by_graph(sequence=[76, 47, 23], graph=graph))
print(validate_by_graph(sequence=[23, 47, 76], graph=graph))

23
None
None


In [6]:
def p1_by_graph(sequences, graph):
    sum = 0
    for sequence in sequences:
        if mid_num := validate_by_graph(sequence, graph):
            sum += mid_num
    return sum

In [7]:
rules, graph, sequences = form_input("./data/day5_small_input_p1.txt")
p1_by_graph(sequences, graph)

143

In [8]:
rules, graph, sequences = form_input("./data/day5_input.txt")
p1_by_graph(sequences, graph)

6041

## Problem 2

This is now correcting the incorrect samples, and returning the sum of middle values of the corrected sequences!

So the idea here, is that for the incorrect sequences, we need to see which rules are involved, and then just...

Actually had a strange idea - since we get all rules explicily and dont have to infer any, it seems we can just correct a sequenece by getting outgoing edges for each element, sorting them by length of the edges list, and thats the correct order? Feels like cheating...

In [9]:
def correct_sequence(sequence, graph):
    sub_graph = {}
    for val in sequence:
        # sub_graph[val] = len(graph[val]) # incorrect! see below!
        sub_graph[val] = len(set(graph[val]) & set(sequence))

    # sort the sub_graph by the number of outgoing edges
    sorted_keys = sorted(sub_graph, key=lambda k: sub_graph[k], reverse=True)
    return sorted_keys[len(sorted_keys) // 2]


def p2(sequences, graph):
    sum = 0
    for sequence in sequences:
        if not validate_by_graph(sequence, graph):
            sum += correct_sequence(sequence, graph)
    return sum

In [10]:
rules, graph, sequences = form_input("./data/day5_small_input_p1.txt")
p2(sequences, graph)

123

In [11]:
rules, graph, sequences = form_input("./data/day5_input.txt")
p2(sequences, graph)

4884

### Error

Note, previously I had:


```python
sub_graph[val] = len(graph[val]) # incorrect! see below!
```

This was incorrect! After debugging on a sequence like:

```
[19, 24, 74, 71, 22, 53, 16, 94, 95, 33, 27, 15, 54, 14, 87, 31, 13, 17, 89]
```

and the subgraphs of `19` and `74`, I realized several had edges of the same length - but not all are involved in this sequence! So instead, we just trim down the edge list to edges involved in the sequence, and then sort by length to get the correct order!